In [ ]:
# Assignment 3: Structural Metrics
# Google Colab Notebook

# Environment Setup and Authentication

# Cloning fork into Google Colab
!git clone https://github.com/emgeek-gs/RS_Analyst_Technical_Assignments.git
#%cd /content/RS_Analyst_Technical_Assignments/structural_metrics

In [2]:
import laspy
import numpy as np
import os
import whitebox
from rasterio import open as rio_open
import pandas as pd
import matplotlib.pyplot as plt

# ---------- USER PARAMETERS ----------
subsample_frac = 0.3          # Fraction of points to retain
clip_fraction = 0.5           # Fraction to clip in each dimension
dtm_resolution = 0.5          # DTM grid resolution in meters (smaller -> higher res)
dsm_resolution = 0.5          # DSM grid resolution in meters (smaller -> higher res)
grid_cells = 100              # Number of rows and columns for zonal grids
figure_dpi = 600              # DPI for saved figures (higher -> finer image)

# Paths and output directory
las_file = "/content/drive/MyDrive/NS/ALS_pointcloud.las"
output_dir = "/content/output/metrics/"
os.makedirs(output_dir, exist_ok=True)

# Load LAS and subsample
las = laspy.read(las_file)
coords = np.vstack((las.x, las.y, las.z)).T
n_sub = int(subsample_frac * len(coords))
idx = np.random.choice(len(coords), n_sub, replace=False)
coords_sub = coords[idx]

# Clip half extents for efficiency
minx, miny = coords_sub[:,0].min(), coords_sub[:,1].min()
maxx, maxy = coords_sub[:,0].max(), coords_sub[:,1].max()
half_w = (maxx - minx) * clip_fraction
half_h = (maxy - miny) * clip_fraction
mask = (
    (coords_sub[:,0] <= minx + half_w) &
    (coords_sub[:,1] <= miny + half_h)
)
las_sub = las[idx][mask]
sub_las = os.path.join(output_dir, "subsampled_clipped.las")
las_sub.write(sub_las)

# Initialize WhiteboxTools
wbt = whitebox.WhiteboxTools()

# Generate high-resolution DTM and DSM
dtm = os.path.join(output_dir, "dtm.tif")
dsm = os.path.join(output_dir, "dsm.tif")
wbt.lidar_tin_gridding(i=sub_las, output=dtm, returns="last", resolution=dtm_resolution)
wbt.lidar_idw_interpolation(i=sub_las, output=dsm, parameter="elevation", returns="last", resolution=dsm_resolution)

# Create Canopy Height Model (CHM)
chm = os.path.join(output_dir, "chm.tif")
wbt.subtract(dsm, dtm, chm)

# Define grid for zonal metrics
nrows = ncols = grid_cells

# Compute bounds and cell size
with rio_open(chm) as src:
    bounds = src.bounds
    cell_w = (bounds.right - bounds.left) / ncols
    cell_h = (bounds.top - bounds.bottom) / nrows

# Calculate metrics per grid cell
results = []
with rio_open(chm) as src:
    for i in range(nrows):
        for j in range(ncols):
            x0 = bounds.left + j * cell_w
            y0 = bounds.bottom + i * cell_h
            x1 = x0 + cell_w
            y1 = y0 + cell_h
            win = src.window(x0, y0, x1, y1).round_offsets().round_lengths()
            data = src.read(1, window=win, masked=True)
            if data.mask.all():
                continue
            vals = data.compressed()
            vals = vals[vals > 0]
            if len(vals) == 0:
                continue
            # Metrics
            mean_h = vals.mean()
            std_h = vals.std()
            p90_h = np.percentile(vals, 90)
            hist, _ = np.histogram(vals, bins=5)
            probs = hist / hist.sum()
            fhd = -np.sum(probs * np.log(probs + 1e-6))
            cc = np.sum(vals > 1) / len(vals)
            results.append([i, j, mean_h, std_h, p90_h, fhd, cc])

# Save metrics to CSV
cols = [
    'row','col','mean_height','std_height','p90_height',
    'foliage_height_diversity','canopy_cover'
]
df = pd.DataFrame(results, columns=cols)
df.to_csv(os.path.join(output_dir, 'metrics.csv'), index=False)

# Visualizations
df = pd.read_csv(os.path.join(output_dir,'metrics.csv'))
p90_grid = df.pivot(index='row', columns='col', values='p90_height')
fhd_grid = df.pivot(index='row', columns='col', values='foliage_height_diversity')
# Create Canopy Cover heatmap
cc_grid = df.pivot(index='row', columns='col', values='canopy_cover')
save_heatmap(cc_grid, 'Canopy Cover (Fraction)', 'canopy_cover_heatmap.png')


# Plotting
def save_heatmap(grid, title, fname):
    plt.figure(figsize=(10,8))
    plt.imshow(grid.values, origin='lower')
    plt.colorbar(label=title)
    plt.title(title)
    plt.xlabel('Column')
    plt.ylabel('Row')
    plt.savefig(os.path.join(output_dir, fname), dpi=figure_dpi)
    plt.close()

save_heatmap(p90_grid, 'P90 Height (m)', 'p90_heatmap.png')
save_heatmap(fhd_grid, 'FHD', 'fhd_heatmap.png')

# Combined subplot
fig, axs = plt.subplots(1,2, figsize=(20,8))
im1 = axs[0].imshow(p90_grid.values, origin='lower')
axs[0].set_title('P90 Height')
im2 = axs[1].imshow(fhd_grid.values, origin='lower')
axs[1].set_title('FHD')
for ax, im in zip(axs, [im1, im2]):
    ax.set_xlabel('Column')
    ax.set_ylabel('Row')
    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
fig.savefig(os.path.join(output_dir,'combined_structure.png'), dpi=figure_dpi)
plt.close()

# Histogram of mean canopy height
plt.figure(figsize=(10,5))
plt.hist(df['mean_height'].dropna(), bins=30)
plt.title('Distribution of Mean Canopy Height')
plt.xlabel('Mean Height (m)')
plt.ylabel('Frequency')
plt.savefig(os.path.join(output_dir,'mean_height_hist.png'), dpi=figure_dpi)
plt.close()

./whitebox_tools --run="LidarTinGridding" --input='/content/output/metrics/subsampled_clipped.las' --output='/content/output/metrics/dtm.tif' --parameter=elevation --returns=last --resolution=0.5 --exclude_cls=7,18 -v --compress_rasters=False

*******************************
* Welcome to LidarTINGridding *
* Powered by WhiteboxTools    *
* www.whiteboxgeo.com         *
*******************************
Performing interpolation...
Reading input LiDAR file...
Reading points: 0%
Reading points: 1%
Reading points: 2%
Reading points: 3%
Reading points: 4%
Reading points: 5%
Reading points: 6%
Reading points: 7%
Reading points: 8%
Reading points: 9%
Reading points: 10%
Reading points: 11%
Reading points: 12%
Reading points: 13%
Reading points: 14%
Reading points: 15%
Reading points: 16%
Reading points: 17%
Reading points: 18%
Reading points: 19%
Reading points: 20%
Reading points: 21%
Reading points: 22%
Reading points: 23%
Reading points: 24%
Reading points: 25%
Reading points: 26%
Reading po